# Searchlight Merged Results Visualization

Meaningful visualization focused on statistically significant results (q ≤ 0.05).


In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import plotting
import matplotlib.pyplot as plt
import seaborn as sns

BASE = Path('/Users/xiaoqianxiao/projects/NARSAD/LSS/searchlight')
DIRS = {
    'ext': BASE / 'ext' / 'merged',
    'rst': BASE / 'rst' / 'merged',
    'dyn_ext': BASE / 'dyn_ext' / 'merged',
    'dyn_rst': BASE / 'dyn_rst' / 'merged',
    'crossphase': BASE / 'crossphase' / 'merged',
}

for k, v in DIRS.items():
    print(f"{k}: {v} (exists={v.exists()})")


## Helpers


In [ ]:
def list_files(folder: Path, pattern: str):
    return sorted(folder.glob(pattern)) if folder.exists() else []


def _is_3d(img):
    return len(img.shape) == 3


def plot_effect_with_q(effect_path: Path, q_path: Path, title: str, cmap: str = 'RdBu_r', q_thresh: float = 0.05):
    effect_img = nib.load(str(effect_path))
    q_img = nib.load(str(q_path))
    if not _is_3d(effect_img) or not _is_3d(q_img):
        print(f"Skip non-3D image: {effect_path.name} or {q_path.name}")
        return None
    effect_data = effect_img.get_fdata()
    q_data = q_img.get_fdata()
    masked = np.where((q_data <= q_thresh) & np.isfinite(q_data), effect_data, 0.0)
    out_img = nib.Nifti1Image(masked, effect_img.affine)
    display = plotting.plot_stat_map(out_img, title=title, display_mode='ortho', threshold=0, cmap=cmap)
    return display


def show_csv(path: Path, n: int = 10):
    if not path.exists():
        print(f"Missing: {path}")
        return None
    df = pd.read_csv(path)
    display(df.head(n))
    print(f"Rows: {len(df)}")
    return df


## EXT / RST: Significant Maps Only


In [ ]:
for key in ['ext', 'rst']:
    folder = DIRS[key]
    if not folder.exists():
        continue
    print(f"\n=== {key.upper()} ===")
    q_files = list_files(folder, '*_q.nii.gz')
    print(f"q maps: {len(q_files)}")
    for q_path in q_files:
        base = q_path.name.replace('_q.nii.gz', '')
        effect_path = None
        for suffix in ['_diff.nii.gz', '_mean.nii.gz']:
            candidate = folder / f"{base}{suffix}"
            if candidate.exists():
                effect_path = candidate
                break
        if effect_path is None:
            continue
        plot_effect_with_q(effect_path, q_path, title=f"{key}: {base}")

    sig_csvs = list_files(folder, '*_sig.csv')
    for csv_path in sig_csvs:
        print(f"\n{csv_path.name}")
        show_csv(csv_path, n=10)

plotting.show()


## Dynamic (EXT/RST): Significant Summary + Top Voxels


In [ ]:
for key in ['dyn_ext', 'dyn_rst']:
    folder = DIRS[key]
    if not folder.exists():
        continue
    print(f"\n=== {key.upper()} ===")

    # Summary counts (per contrast)
    summary_path = folder / 'dynamic_summary_contrasts.csv'
    if summary_path.exists():
        print("\nDynamic summary contrasts")
        show_csv(summary_path, n=10)

    # Top significant voxels
    sig_path = folder / 'dynamic_sig_merged.csv'
    if sig_path.exists():
        print("\nDynamic significant voxels")
        df = pd.read_csv(sig_path)
        display(df.head(10))
        print(f"Rows: {len(df)}")

    # Plot only if q maps exist
    q_files = list_files(folder, '*_q.nii.gz')
    for q_path in q_files:
        base = q_path.name.replace('_q.nii.gz', '')
        effect_path = None
        for suffix in ['_diff.nii.gz', '_mean.nii.gz']:
            candidate = folder / f"{base}{suffix}"
            if candidate.exists():
                effect_path = candidate
                break
        if effect_path is None:
            continue
        plot_effect_with_q(effect_path, q_path, title=f"{key}: {base}")

plotting.show()


## Cross-phase: Significant Summary + Top Voxels


In [ ]:
folder = DIRS['crossphase']
if folder.exists():
    summary_path = folder / 'crossphase_summary_contrasts.csv'
    if summary_path.exists():
        print("\nCross-phase summary contrasts")
        show_csv(summary_path, n=10)

    sig_path = folder / 'crossphase_sig_merged.csv'
    if sig_path.exists():
        print("\nCross-phase significant voxels")
        df = pd.read_csv(sig_path)
        display(df.head(10))
        print(f"Rows: {len(df)}")

    q_files = list_files(folder, '*_q.nii.gz')
    for q_path in q_files:
        base = q_path.name.replace('_q.nii.gz', '')
        effect_path = None
        for suffix in ['_diff.nii.gz', '_mean.nii.gz']:
            candidate = folder / f"{base}{suffix}"
            if candidate.exists():
                effect_path = candidate
                break
        if effect_path is None:
            continue
        plot_effect_with_q(effect_path, q_path, title=f"crossphase: {base}")

plotting.show()


## Optional: Only plot significant maps for selected prefixes


In [ ]:
# Set to a list of prefixes to limit plots, or leave empty to skip
PREFIXES = []

if PREFIXES:
    for key, folder in DIRS.items():
        if not folder.exists():
            continue
        q_files = [p for p in list_files(folder, '*_q.nii.gz') if any(p.name.startswith(px) for px in PREFIXES)]
        for q_path in q_files:
            base = q_path.name.replace('_q.nii.gz', '')
            effect_path = None
            for suffix in ['_diff.nii.gz', '_mean.nii.gz']:
                candidate = folder / f"{base}{suffix}"
                if candidate.exists():
                    effect_path = candidate
                    break
            if effect_path is None:
                continue
            plot_effect_with_q(effect_path, q_path, title=f"{key}: {base}")

plotting.show()
